## LangChain 

### Environment

In [1]:
import os
from langchain_openai import ChatOpenAI
from langchain_core.messages import AIMessage, HumanMessage, SystemMessage, trim_messages
from langchain_core.chat_history import (
    BaseChatMessageHistory, # A messages class for message-list management. messages()
    InMemoryChatMessageHistory, # Messages class on top of BaseMessage
)
from langchain_core.runnables.history import RunnableWithMessageHistory  # runnable + messages

with open('../key.txt', "r", encoding="utf-8") as file:
    content = file.readline()
    
os.environ['LANGCHAIN_TRACING_V2'] = "true"
os.environ['LANGCHAIN_API_KEY'] = content.split(' ')[-1]
os.environ['LANGCHAIN_ENDPOINT'] = "https://api.smith.langchain.com"
os.environ['LANGCHAIN_PROJECT'] = "project-test-1"

### Base runnable invoke

In [2]:
model = ChatOpenAI(model_name = 'gpt-4o-mini')
model.invoke(
    [
        HumanMessage(content="Hi! I'm Bob"), # 1. Message
        AIMessage(content="Hello Bob! How can I assist you today?"),
        HumanMessage(content="What's my name?"), # shows that the thread is remembered
    ]
)

AIMessage(content='Your name is Bob! How can I help you today, Bob?', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 14, 'prompt_tokens': 33, 'total_tokens': 47, 'prompt_tokens_details': {'cached_tokens': 0}, 'completion_tokens_details': {'reasoning_tokens': 0}}, 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_e2bde53e6e', 'finish_reason': 'stop', 'logprobs': None}, id='run-33498634-ba6a-4a66-8efa-c10ca65d66bf-0', usage_metadata={'input_tokens': 33, 'output_tokens': 14, 'total_tokens': 47, 'input_token_details': {'cache_read': 0}, 'output_token_details': {'reasoning': 0}})

### Runnable+MessageHistory invoke
Messages:
- 1. message = HumanMessage/AIMessage/HumanMessage/trim_messages
- 2. messages = [message]
  3. ChatMessageHistory:  MessageOps() + messages
  4. Prompt = template/builder. Prompt.invoke(variable) = messages.
  5. trimmer : trimmer.invoke(messages) = messages

Caller:
- 1. model: model.invoke(messages)
  2. runnableWithHistory: model.invoke(messages, MessageHistory())
  3. chain = messages -> prompt | model
  4. runnablePassThrough: .assign(key:value) -> output[key]=value

In [3]:
# 1. A function to manage sessions. 'id'->'history'
#   - InMemoryChatMessageHistory() : init a session
store = {}
def get_session_history(session_id: str) -> BaseChatMessageHistory: 
    if session_id not in store:
        store[session_id] = InMemoryChatMessageHistory()
    return store[session_id]

# 2. A function to manage 'id'->'history'
runnable_with_message_history = RunnableWithMessageHistory(model, get_session_history)

# 3. Invoke by passing session-id; the runnable class Auto-manage session messages 
config = {"configurable": {"session_id": "abc2"}}
response = runnable_with_message_history.invoke(
    [HumanMessage(content="I have a black car, but it's dirty now")],
    config=config,
)
print(len(store['abc2'].messages), len(response.content))

2 1060


### Prompt Management

In [11]:
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder

## 1. creates a template. 
prompt = ChatPromptTemplate.from_messages([
        (
            "system",
            "You are a helpful assistant. Answer all questions to the best of your ability. Answer only in {Lan}.",
        ),
        MessagesPlaceholder(variable_name="messages"), 
    ])
# prompt.format(Lan = 'Chinese', messages = store['abc2'].messages) # prompt -> str
# prompt.invoke({Lan = 'Chinese', messages = store['abc2'].messages}) # prompt -> promptValue

## 2. chain = prompt & invoke
chain = prompt | model # both (prompt & model) have .invoke | is overloaded

# chain.invoke( {"messages": [HumanMessage(content="hi! I'm bob")], "Lan": "Chinese"})
# model.invoke(prompt.format(Lan = 'Chinese', messages = store['abc2'].messages)) # call by message


AIMessage(content='你好，Bob！有什么我可以帮助你的吗？', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 11, 'prompt_tokens': 36, 'total_tokens': 47, 'prompt_tokens_details': {'cached_tokens': 0}, 'completion_tokens_details': {'reasoning_tokens': 0}}, 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_e2bde53e6e', 'finish_reason': 'stop', 'logprobs': None}, id='run-15cac590-0549-4f4b-bf47-2724659a0a66-0', usage_metadata={'input_tokens': 36, 'output_tokens': 11, 'total_tokens': 47, 'input_token_details': {'cache_read': 0}, 'output_token_details': {'reasoning': 0}})

In [68]:
## 3. runnable = chain & history
with_message_history = RunnableWithMessageHistory(chain, 
                                                  get_session_history,
                                                  input_messages_key="messages")

with_message_history.invoke(
    {"messages": [HumanMessage(content="How many words have we talked already?")], "Lan": "Chinese"},
    config = {"configurable": {"session_id": "abc2"}},
)

AIMessage(content='We have exchanged 5 messages so far. How can I assist you further, Bob?', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 18, 'prompt_tokens': 272, 'total_tokens': 290, 'prompt_tokens_details': {'cached_tokens': 0}, 'completion_tokens_details': {'reasoning_tokens': 0}}, 'model_name': 'gpt-3.5-turbo-0125', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None}, id='run-11555d21-4886-4bca-a6f5-e38a87bb32e3-0', usage_metadata={'input_tokens': 272, 'output_tokens': 18, 'total_tokens': 290, 'input_token_details': {'cache_read': 0}, 'output_token_details': {'reasoning': 0}})

In [ ]:
## 4. chain + trimmer, runnable + chain
from langchain_core.messages import SystemMessage, trim_messages
from langchain_core.runnables import RunnablePassthrough
from operator import itemgetter

trimmer = trim_messages(
    max_tokens=65,
    strategy="last",
    token_counter=model,
    include_system=True,
    allow_partial=False,
    start_on="human",
)

messages = [
    SystemMessage(content="you're a good assistant"),
    HumanMessage(content="hi! I'm bob"),
    AIMessage(content="hi!"),
    HumanMessage(content="I like vanilla ice cream"),
    AIMessage(content="nice"),
    HumanMessage(content="whats 2 + 2"),
    AIMessage(content="4"),
    HumanMessage(content="thanks"),
    AIMessage(content="no problem!"),
    HumanMessage(content="having fun?"),
    AIMessage(content="yes!"),
]

## trimmer.invoke(messages)

## Order: in={in} -> (itemgetter)in={in}['messages'] -> out=trimmer(in) -> assign({messages=out, in={in})
chain = (RunnablePassthrough.assign(messages=(itemgetter("messages") | trimmer))
         | prompt | model)
# chain.invoke({"messages": messages, "Lan": "Chinese"})

## chain -> runnable_history. Order: 'message':get_history() + invoke_message() -> chain
runnable = RunnableWithMessageHistory(chain, 
                                      get_session_history,
                                      input_messages_key="messages")

runnable.invoke({"messages": [HumanMessage(content="How to write a good project proposal?")], "Lan": "Chinese"},
                config = {"configurable": {"session_id": "abc2"}})

## LangGraph

In [15]:
from langgraph.checkpoint.memory import MemorySaver
from langgraph.graph import START, MessagesState, StateGraph
from typing import Sequence

from langchain_core.messages import BaseMessage
from langgraph.graph.message import add_messages
from typing_extensions import Annotated, TypedDict

# model
model = ChatOpenAI(model_name = 'gpt-4o-mini')

# Define a new graph
workflow = StateGraph(state_schema=MessagesState) # Just a dict('messages')

# Define the function that calls the model
def call_model(state: MessagesState): # 
    response = model.invoke(state["messages"]) 
    return {"messages": response}

# Define the (single) node in the graph
workflow.add_edge(START, "model")
workflow.add_node("model", call_model)

# Add memory
memory = MemorySaver()
app = workflow.compile(checkpointer=memory)

In [ ]:
config = {"configurable": {"thread_id": "user-1"}}

query = "Hi! I'm Bob."
input_messages = [HumanMessage(query)]
output = app.invoke({"messages": input_messages}, config)
output["messages"][-1].pretty_print()  # output contains all messages in state

### Prompt + Graph

In [39]:
from typing import Sequence
from langchain_core.messages import BaseMessage
from langgraph.graph.message import add_messages
from typing_extensions import Annotated, TypedDict
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder

## 1. creates a template. 
prompt = ChatPromptTemplate.from_messages([
        (
            "system",
            "You are a helpful assistant. Answer all questions to the best of your ability. Answer only in {Lan}.",
        ),
        MessagesPlaceholder(variable_name="messages"), 
    ])

## 2. call_model = prompt | model 
class State(TypedDict):
    messages: Annotated[Sequence[BaseMessage], add_messages]
    Lan: str

def call_model(state: State): # call model input = {'messages', 'Lan'}
    chain = prompt | model
    response = chain.invoke(state)
    return {"messages": [response]}

workflow = StateGraph(state_schema=State)
workflow.add_edge(START, "model")
workflow.add_node("model", call_model)

memory = MemorySaver()
app = workflow.compile(checkpointer=memory)

In [ ]:
config = {"configurable": {"thread_id": "abc456"}}
query = "What's the weather like today"
language = "Chinese"

input_messages = [HumanMessage(query)]
output = app.invoke(
    {"messages": input_messages, "Lan": language},
    config,
)
# [ m.pretty_print() for m in output["messages"] ]
 output["messages"][-1].pretty_print()

In [43]:
config = {"configurable": {"thread_id": "abc456"}}
query = "Which openAI model support image plotting"
language = "Chinese"

input_messages = [HumanMessage(query)]

for chunk, metadata in app.stream(
    {"messages": input_messages, "language": language},
    config,
    stream_mode="messages",
):
    if isinstance(chunk, AIMessage):  # Filter to just model responses
        print(chunk.content, end="|")

|支持|图|像|生成|和|绘|制|的|Open|AI|模型|是|D|ALL|-E|。|D|ALL|-E|可以|生成|图|像|，|基|于|文本|描述|创造|视觉|内容|。如果|你|还有|其他|问题|或|需要|进一步|的信息|，请|告诉|我|！||

### Plot VizGraph

In [47]:
def generate_dot(graph):
    dot_lines = ["digraph G {"]
    for node in graph.nodes:
        dot_lines.append(f'    "{node}";')
    for edge in graph.edges:
        dot_lines.append(f'    "{edge}" -> "{edge[1]}";')
    dot_lines.append("}")
    return "\n".join(dot_lines)

# Generate DOT representation
dot_representation = generate_dot(workflow)

with open("workflow.dot", "w") as f:
    f.write(dot_representation)

# dot -Tpng workflow.dot -o workflow.png